In [ ]:
import matplotlib.pyplot as plt
import numpy as np ,pandas as pd
from collections import OrderedDict,Counter
import os
import statsmodels.stats.multitest as multi
import scipy.stats as stats
from scipy.stats import pearsonr
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import math

In [ ]:
dir="E:\\project\\singcells\\3_function_analyse\\"
dir1="E:\\project\\singcells\\SCENIC\\jt_4cancer_scenic_result\\"
dir2="E:\\project\\A_normal_infor\\MsigDB\\cancer_hallmark\\"

In [ ]:
tar=pd.read_csv(dir1+"4_cancer_reglons_targetgene.txt",header=0,index_col=0,sep="\t")

In [ ]:
tar.head(3)

In [ ]:
cacner_hallmark=OrderedDict()

r=open(dir2+"h.all.v7.2.symbols.xls")

line=r.readline()
while line:
    arr=line.strip().split("\t")

    cacner_hallmark[arr[0]]="\t".join(arr[2:])
    line=r.readline()
r.close()

In [ ]:
c_r=[]
im_p=[]
p_v=[]
inter_gene=[]
all_n=19954

In [ ]:
for ind,cols in tar.iterrows():
    reg_ta_gene=cols[2].split(",")
    
    for k,v in cacner_hallmark.items():
        c_r.append(ind+"_"+cols[0]+"_"+cols[1])
        im_p.append(k)
        cancer_hallmark_gene=v.split("\t")
        inter=list(set(cancer_hallmark_gene).intersection(set(reg_ta_gene)))
        if len(inter)>3:
            p=stats.hypergeom.sf(len(inter)-1,all_n,len(cancer_hallmark_gene),len(reg_ta_gene))
            p_v.append(p)
            inter_gene.append(",".join(inter))
        else:
            p_v.append("-")
            inter_gene.append("-")
            

In [ ]:
hypergeom_re=pd.DataFrame()

hypergeom_re["cancer_celltype_regulon"]=c_r
hypergeom_re["cacer_hallmark"]=im_p
hypergeom_re["p_value"]=p_v
hypergeom_re["inter_gene"]=inter_gene

In [ ]:
hypergeom_re.head(3)

In [ ]:
hypergeom_re.to_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom.txt",header=True,sep="\t",index=False)

In [ ]:
data_on=hypergeom_re[hypergeom_re["p_value"]!="-"]
data_on.shape

In [ ]:
data_on.to_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom_on.txt",header=True,sep="\t",index=False)

In [ ]:
df_p=data_on[data_on["p_value"]<0.05]

In [ ]:
df_p.to_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom_on_sig(p005).txt",header=True,sep="\t",index=False)

In [ ]:
len(set(data_on["cacer_hallmark"]))

In [ ]:
len(set(df_p["cacer_hallmark"]))

In [ ]:
data_fdr=pd.read_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom_on_fdr.txt",header=0,sep="\t")

In [ ]:
data_fdr.head(3)

In [ ]:
df_p=data_fdr[data_fdr["fdr"]<0.001]

In [ ]:
df_p.head(3)

In [ ]:
cancer_all=[]
celltype_all=[]
regulons_all=[]
cancer_hallmark=[]
gene_all=[]


In [ ]:
for ind,cols in df_p.iterrows():
    for ww in cols[-2].split(","):
        cancer_all.append(cols[0].split("_")[0])
        celltype_all.append(cols[0].split("_")[1])
        regulons_all.append(cols[0].split("_")[-1])
        cancer_hallmark.append(cols[1])
        gene_all.append(ww)

In [ ]:
rbp_cancer_hallmark_infor=pd.DataFrame()

rbp_cancer_hallmark_infor["cancer"]=cancer_all
rbp_cancer_hallmark_infor["celltype"]=celltype_all
rbp_cancer_hallmark_infor["regulons"]=regulons_all
rbp_cancer_hallmark_infor["cancer_hallmark"]=cancer_hallmark
rbp_cancer_hallmark_infor["genes"]=gene_all

In [ ]:
rbp_cancer_hallmark_infor.head(3)

In [ ]:
rbp_cancer_hallmark_infor.to_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom_on_sig(fdr0.001)_link.txt",header=True,sep="\t",index=False)

In [ ]:
rbp_cancer_hallmark_infor.index=rbp_cancer_hallmark_infor["cancer"].tolist()
data=rbp_cancer_hallmark_infor

In [ ]:
data_genes=list(set(data["genes"]))

data_cancer_hallmark=list(set(data["cancer_hallmark"]))

data_reg_bc=list(set(data.loc["BC"]["regulons"]))
data_reg_lc=list(set(data.loc["LC"]["regulons"]))
data_reg_crc=list(set(data.loc["CRC"]["regulons"]))
data_reg_ovc=list(set(data.loc["OvC"]["regulons"]))

In [ ]:
len(data_genes)
len(data_cancer_hallmark)
len(data_reg_bc)
len(data_reg_lc)
len(data_reg_crc)
len(data_reg_ovc)
data.shape

In [ ]:
gene_2c_h=[]
for gene0 in data_genes:
    if len(set(data[data["genes"]==gene0]["cancer_hallmark"].tolist()))>=2:
        gene_2c_h.append(gene0)
len(gene_2c_h)
data_2=data[data["genes"].isin(gene_2c_h)]

In [ ]:
data_2.to_csv("E:\\project\\singcells\\3_function_analyse\\cancer_hallmark_function_analyse\\rbp_cancer_hallmark_hypergeom_on_sig(fdr0.001)_2patheway_infor.txt",header=True,sep="\t",index=False)

In [ ]:
data_genes=list(set(data_2["genes"]))

data_cancer_hallmark=list(set(data_2["cancer_hallmark"]))

data_reg_bc=list(set(data_2.loc["BC"]["regulons"]))
data_reg_lc=list(set(data_2.loc["LC"]["regulons"]))
data_reg_crc=list(set(data_2.loc["CRC"]["regulons"]))
data_reg_ovc=list(set(data_2.loc["OvC"]["regulons"]))

In [ ]:
len(data_genes)
len(data_cancer_hallmark)
len(data_reg_bc)
len(data_reg_lc)
len(data_reg_crc)
len(data_reg_ovc)

In [449]:
def color_change(value):
  digit = list(map(str, range(10))) + list("ABCDEF")
  if isinstance(value, tuple):
    string = '#'
    for i in value:
      a1 = i // 16
      a2 = i % 16
      string += digit[a1] + digit[a2]
    return string
  elif isinstance(value, str):
    a1 = digit.index(value[1]) * 16 + digit.index(value[2])
    a2 = digit.index(value[3]) * 16 + digit.index(value[4])
    a3 = digit.index(value[5]) * 16 + digit.index(value[6])
    return (a1, a2, a3)

In [457]:
color_change("#d98d2c".upper())

(217, 141, 44)

In [475]:

for ww in data_reg_lc:
    data_text=data_2[(data_2["cancer"]=="LC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first')
    if data_text.shape[0] ==2:
        print(data_text)
        

   cancer    celltype       regulons          cancer_hallmark   genes
LC     LC  Fibroblast  HNRNPF (537g)  HALLMARK_MYC_TARGETS_V1   HSPD1
LC     LC           T  HNRNPF (537g)  HALLMARK_MYC_TARGETS_V1  NAP1L1


In [477]:
data_2[(data_2["cancer"]=="LC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first').iloc[0,1]

'Epithelial'

In [479]:
color_celltype=pd.read_csv("E:\\project\\singcells\\celltype_colors.txt",header=None,sep="\t")

In [483]:
color_celltype.head(3)

,0,1,2
0,BC,B_cell,#d98d2c
1,BC,Mast,#7f88c2
2,BC,Cancer,#bca131


In [487]:
hs=[]
st=[]
end=[]
sy=[]
color=[]

In [488]:
###cancers###

###bc
sp=3
nu=0
for ww in data_reg_bc:
    hs.append("hs1")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    cell_type0=data_2[(data_2["cancer"]=="BC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first').iloc[0,1]
    if cell_type0=="B":
        color0=color_celltype[(color_celltype[0]=="BC")&(color_celltype[1]=="B_cell")].iloc[0,2]
    elif cell_type0=="T":
        color0=color_celltype[(color_celltype[0]=="BC")&(color_celltype[1]=="T_cell")].iloc[0,2]
    else:
        color0=color_celltype[(color_celltype[0]=="BC")&(color_celltype[1]==cell_type0)].iloc[0,2]
    color_rgb=color_change(color0.upper())
    color.append("color="+str(color_rgb[0])+","+str(color_rgb[1])+","+str(color_rgb[2]))
###crc
sp=3
nu=0
for ww in data_reg_crc:
    hs.append("hs2")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    cell_type0=data_2[(data_2["cancer"]=="CRC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first').iloc[0,1]
    if cell_type0=="B":
        color0=color_celltype[(color_celltype[0]=="CRC")&(color_celltype[1]=="B_cell")].iloc[0,2]
    elif cell_type0=="T":
        color0=color_celltype[(color_celltype[0]=="CRC")&(color_celltype[1]=="T_cell")].iloc[0,2]
    elif cell_type0=="Mast":
        color0=color_celltype[(color_celltype[0]=="CRC")&(color_celltype[1]=="Mast_cell")].iloc[0,2]
    else:
        color0=color_celltype[(color_celltype[0]=="CRC")&(color_celltype[1]==cell_type0)].iloc[0,2]
    color_rgb=color_change(color0.upper())
    color.append("color="+str(color_rgb[0])+","+str(color_rgb[1])+","+str(color_rgb[2]))

###lc
sp=3
nu=0
for ww in data_reg_lc:
    hs.append("hs3")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    
    cell_type0=data_2[(data_2["cancer"]=="LC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first').iloc[0,1]
    if cell_type0=="B":
        color0=color_celltype[(color_celltype[0]=="LC")&(color_celltype[1]=="B_cell")].iloc[0,2]
    elif cell_type0=="T":
        color0=color_celltype[(color_celltype[0]=="LC")&(color_celltype[1]=="T_cell")].iloc[0,2]
    elif cell_type0=="Mast":
        color0=color_celltype[(color_celltype[0]=="CRC")&(color_celltype[1]=="Mast_cell")].iloc[0,2]
    else:
        color0=color_celltype[(color_celltype[0]=="LC")&(color_celltype[1]==cell_type0)].iloc[0,2]
    color_rgb=color_change(color0.upper())
    color.append("color="+str(color_rgb[0])+","+str(color_rgb[1])+","+str(color_rgb[2]))

###ovc
sp=3
nu=0
for ww in data_reg_ovc:
    hs.append("hs4")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    cell_type0=data_2[(data_2["cancer"]=="OvC")&(data_2["regulons"]==ww)].drop_duplicates(subset=['celltype', 'regulons'], keep='first').iloc[0,1]
    if cell_type0=="B":
        color0=color_celltype[(color_celltype[0]=="OvC")&(color_celltype[1]=="B_cell")].iloc[0,2]
    elif cell_type0=="T":
        color0=color_celltype[(color_celltype[0]=="OvC")&(color_celltype[1]=="T_cell")].iloc[0,2]
    else:
        color0=color_celltype[(color_celltype[0]=="OvC")&(color_celltype[1]==cell_type0)].iloc[0,2]
    color_rgb=color_change(color0.upper())
    color.append("color="+str(color_rgb[0])+","+str(color_rgb[1])+","+str(color_rgb[2]))



In [489]:
cell_type0

'EC'

In [490]:
###cancer_hallmark

sp=10
nu=0
for ww in data_cancer_hallmark:
    hs.append("hs5")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    color.append("color=15,15,15")


In [491]:
###genes

sp=4
nu=0
for ww in data_genes:
    hs.append("hs6")
    st.append(nu)
    end.append(nu+sp)
    nu=end[-1]+1
    sy.append("".join(ww.split(" ")))
    color.append("color=15,15,15")

In [492]:
labels_all=pd.DataFrame()
labels_all["hs"]=hs
labels_all["st"]=st
labels_all["end"]=end
labels_all["sy"]=sy
labels_all["color"]=color


In [493]:
dir_circos="D:\\Circos\\circos\\circos-0.68-1\\circos-tutorials-0.67\\tutorials\\zww\\SCENIC\\function_cancer_hallmark\\circos\\base\\"

In [494]:
labels_all.to_csv(dir_circos+"labels_all.txt",header=False,index=False,sep="\t")

In [ ]:
hs=[]
st=[]
end=[]
sy=[]
hs1=[]
st1=[]
end1=[]
sy1=[]

In [ ]:
###genes

sp=4
nu=0
i=0
for ww in data_genes:
    i=i+1
    if i%2 ==0:
        hs.append("hs6")
        st.append(nu)
        end.append(nu+sp)
        nu=end[-1]+1
        sy.append("".join(ww.split(" ")))
    else:
        hs1.append("hs6")
        st1.append(nu)
        end1.append(nu+sp)
        nu=end1[-1]+1
        sy1.append("".join(ww.split(" ")))
        

In [ ]:
labels_gene=pd.DataFrame()
labels_gene["hs"]=hs
labels_gene["st"]=st
labels_gene["end"]=end
labels_gene["sy"]=sy
labels_gene1=pd.DataFrame()
labels_gene1["hs"]=hs1
labels_gene1["st"]=st1
labels_gene1["end"]=end1
labels_gene1["sy"]=sy1

In [ ]:
labels_gene.to_csv(dir_circos+"labels_gene.txt",header=False,index=False,sep="\t")
labels_gene1.to_csv(dir_circos+"labels_gene1.txt",header=False,index=False,sep="\t")

In [ ]:
####connector ####

In [ ]:
hs=[]
st=[]
end=[]

In [ ]:
data_connector=labels[labels["hs"]=="hs6"]

In [ ]:
for ind,cols in data_connector.iterrows():
    hs.append(cols[0])
    st.append(int((cols[1]+cols[2])/2))
    end.append(int((cols[1]+cols[2])/2))

In [ ]:
connector=pd.DataFrame()

connector["hs"]=hs
connector["st"]=st
connector["end"]=end

In [ ]:
connector.to_csv(dir_circos+"connector.txt",header=False,index=False,sep="\t")

In [ ]:
hs=[]
st=[]
end=[]
for ind,cols in labels_gene.iterrows():
    hs.append(cols[0])
    st.append(int((cols[1]+cols[2])/2))
    end.append(int((cols[1]+cols[2])/2))
connector1=pd.DataFrame()

connector1["hs"]=hs
connector1["st"]=st
connector1["end"]=end

In [ ]:
connector1.to_csv(dir_circos+"connector1.txt",header=False,index=False,sep="\t")

In [ ]:
hs=[]
st=[]
end=[]
for ind,cols in labels_gene1.iterrows():
    hs.append(cols[0])
    st.append(int((cols[1]+cols[2])/2))
    end.append(int((cols[1]+cols[2])/2))
connector2=pd.DataFrame()

connector2["hs"]=hs
connector2["st"]=st
connector2["end"]=end

In [ ]:
connector2.to_csv(dir_circos+"connector2.txt",header=False,index=False,sep="\t")

In [ ]:
######link#####

In [ ]:
g_hs_all=[]
g_st_all=[]
g_end_all=[]
tar_hs=[]
tar_st=[]
tar_end=[]
colors=[]

In [ ]:
for gene0 in data_genes:
    g_hs=labels_all[labels_all["sy"]==gene0].iloc[0,0]
    g_st=labels_all[labels_all["sy"]==gene0].iloc[0,1]
    g_end=labels_all[labels_all["sy"]==gene0].iloc[0,2]


    data0=data_2[data_2["genes"]==gene0]

    if "BC" in data0.index:
        for ind,cols in data0[data0["cancer"]=="BC"].iterrows():
            reg0="".join(cols[2].split(" "))
            reg0_hs=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs1")].iloc[0,0]
            reg0_st=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs1")].iloc[0,1]
            reg0_end=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs1")].iloc[0,2]
            g_hs_all.append(g_hs)
            g_st_all.append(g_st)
            g_end_all.append(g_end)
            tar_hs.append(reg0_hs)
            tar_st.append(reg0_st)
            tar_end.append(reg0_end)
            colors.append("color=233,105,101")

    if "CRC" in data0.index:
        for ind,cols in data0[data0["cancer"]=="CRC"].iterrows():
            reg0="".join(cols[2].split(" "))
            reg0_hs=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs2")].iloc[0,0]
            reg0_st=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs2")].iloc[0,1]
            reg0_end=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs2")].iloc[0,2]
            g_hs_all.append(g_hs)
            g_st_all.append(g_st)
            g_end_all.append(g_end)
            tar_hs.append(reg0_hs)
            tar_st.append(reg0_st)
            tar_end.append(reg0_end)
            colors.append("color=112,161,40")

    if "LC" in data0.index:
        for ind,cols in data0[data0["cancer"]=="LC"].iterrows():
            reg0="".join(cols[2].split(" "))
            reg0_hs=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs3")].iloc[0,0]
            reg0_st=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs3")].iloc[0,1]
            reg0_end=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs3")].iloc[0,2]
            g_hs_all.append(g_hs)
            g_st_all.append(g_st)
            g_end_all.append(g_end)
            tar_hs.append(reg0_hs)
            tar_st.append(reg0_st)
            tar_end.append(reg0_end)
            colors.append("color=0,171,183")

    if "OvC" in data0.index:
        for ind,cols in data0[data0["cancer"]=="OvC"].iterrows():
            reg0="".join(cols[2].split(" "))
            reg0_hs=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs4")].iloc[0,0]
            reg0_st=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs4")].iloc[0,1]
            reg0_end=labels_all[(labels_all["sy"]==reg0)&(labels_all["hs"]=="hs4")].iloc[0,2]
            g_hs_all.append(g_hs)
            g_st_all.append(g_st)
            g_end_all.append(g_end)
            tar_hs.append(reg0_hs)
            tar_st.append(reg0_st)
            tar_end.append(reg0_end)
            colors.append("color=159,122,180")

In [ ]:
link_gene=pd.DataFrame()

link_gene["g_hs"]=g_hs_all
link_gene["g_st_all"]=g_st_all
link_gene["g_end_all"]=g_end_all
link_gene["tar_hs"]=tar_hs
link_gene["tar_st"]=tar_st
link_gene["tar_end"]=tar_end
link_gene["colors"]=colors

In [ ]:
link_gene.to_csv(dir_circos+"link_gene.txt",header=False,index=False,sep="\t")

In [ ]:
######pathway-regulons####

In [ ]:
data_2.head(3)

In [ ]:
zd_ch_reg_cancer=OrderedDict({})

In [ ]:
for ind,cols in data_2.iterrows():
    zd_ch_reg_cancer[cols[0]+"\t"+("".join(cols[2].split(" ")))+"\t"+cols[3]]=cols[-1]

In [ ]:
p_hs_all=[]
p_st_all=[]
p_end_all=[]
reg_hs=[]
reg_st=[]
reg_end=[]
colors_imm=[]

In [ ]:
for k,v in zd_ch_reg_cancer.items():
    arr=k.split("\t")
    reg0=arr[1]
    data_reg=labels_all[labels_all["sy"]==reg0]
    for i in range(0,data_reg.shape[0]):
        g_hs=labels_all[labels_all["sy"]==arr[1]].iloc[i,0]
        g_st=labels_all[labels_all["sy"]==arr[1]].iloc[i,1]
        g_end=labels_all[labels_all["sy"]==arr[1]].iloc[i,2]
        reg0_hs=labels_all[labels_all["sy"]==arr[-1]].iloc[0,0]
        reg0_st=labels_all[labels_all["sy"]==arr[-1]].iloc[0,1]
        reg0_end=labels_all[labels_all["sy"]==arr[-1]].iloc[0,2]
        p_hs_all.append(g_hs)
        p_st_all.append(g_st)
        p_end_all.append(g_end)
        reg_hs.append(reg0_hs)
        reg_st.append(reg0_st)
        reg_end.append(reg0_end)

        if data_reg["hs"].tolist()[i]=="hs1":
            colors_imm.append("color=233,105,101")
        elif data_reg["hs"].tolist()[i]=="hs2":
            colors_imm.append("color=112,161,40")
        elif data_reg["hs"].tolist()[i]=="hs3":
            colors_imm.append("color=0,171,183")
        elif data_reg["hs"].tolist()[i]=="hs4":
            colors_imm.append("color=159,122,180")


In [ ]:
link_p_g=pd.DataFrame()

link_p_g["g_hs"]=p_hs_all
link_p_g["g_st_all"]=p_st_all
link_p_g["g_end_all"]=p_end_all
link_p_g["tar_hs"]=reg_hs
link_p_g["tar_st"]=reg_st
link_p_g["tar_end"]=reg_end
link_p_g["colors"]=colors_imm

In [ ]:
link_p_g.to_csv(dir_circos+"link_p_r.txt",header=False,index=False,sep="\t")

In [ ]:
#####pathway-gene#####

In [ ]:
p_hs_all=[]
p_st_all=[]
p_end_all=[]
reg_hs=[]
reg_st=[]
reg_end=[]
colors_imm=[]

In [ ]:
data_2.head(3)

In [ ]:
for ind,cols in data_2.iterrows():
   
    g_hs=labels_all[labels_all["sy"]==cols[-1]].iloc[0,0]
    g_st=labels_all[labels_all["sy"]==cols[-1]].iloc[0,1]
    g_end=labels_all[labels_all["sy"]==cols[-1]].iloc[0,2]
    reg0_hs=labels_all[labels_all["sy"]==cols[-2]].iloc[0,0]
    reg0_st=labels_all[labels_all["sy"]==cols[-2]].iloc[0,1]
    reg0_end=labels_all[labels_all["sy"]==cols[-2]].iloc[0,2]
    p_hs_all.append(g_hs)
    p_st_all.append(g_st)
    p_end_all.append(g_end)
    reg_hs.append(reg0_hs)
    reg_st.append(reg0_st)
    reg_end.append(reg0_end)
    if cols[0]=="BC":
        colors_imm.append("color=233,105,101")
    elif cols[0]=="CRC":
        colors_imm.append("color=112,161,40")
    elif cols[0]=="LC":
        colors_imm.append("color=0,171,183")
    elif cols[0]=="OvC":
        colors_imm.append("color=159,122,180")

In [ ]:
link_imm_gene=pd.DataFrame()

link_imm_gene["g_hs"]=p_hs_all
link_imm_gene["g_st_all"]=p_st_all
link_imm_gene["g_end_all"]=p_end_all
link_imm_gene["tar_hs"]=reg_hs
link_imm_gene["tar_st"]=reg_st
link_imm_gene["tar_end"]=reg_end
link_imm_gene["colors"]=colors_imm

In [ ]:
link_imm_gene.to_csv(dir_circos+"link_p_g.txt",header=False,index=False,sep="\t")

In [ ]:
####### labels_4_cancer######

In [495]:
labels_4_cancer=pd.read_csv(dir_circos+"labels_4_cancer.txt",header=None,sep="\t")

In [500]:
hs=[]
st=[]
end=[]
sy=[]
color=[]
hs1=[]
st1=[]
end1=[]
sy1=[]
color1=[]

In [501]:
i=0
for ind,cols in labels_4_cancer.iterrows():
    i=i+1
    if i%2 ==0:
        hs.append(cols[0])
        st.append(cols[1])
        end.append(cols[2])
        sy.append(cols[3].split("(")[0])
        color.append(cols[4])
    else:
        hs1.append(cols[0])
        st1.append(cols[1])
        end1.append(cols[2])
        sy1.append(cols[3].split("(")[0])
        color1.append(cols[4])
        

In [502]:
labels_gene=pd.DataFrame()
labels_gene["hs"]=hs
labels_gene["st"]=st
labels_gene["end"]=end
labels_gene["sy"]=sy
labels_gene["color"]=color
labels_gene1=pd.DataFrame()
labels_gene1["hs"]=hs1
labels_gene1["st"]=st1
labels_gene1["end"]=end1
labels_gene1["sy"]=sy1
labels_gene1["color"]=color1

In [503]:
labels_gene.to_csv(dir_circos+"labels_4_cancer1_new.txt",header=False,index=False,sep="\t")
labels_gene1.to_csv(dir_circos+"labels_4_cancer2_new.txt",header=False,index=False,sep="\t")

In [ ]:
zd_ch_reg_cancer["OvC"]

In [ ]:
labels_gene.head(3)

In [ ]:
####connector1#####

In [ ]:
hs=[]
st=[]
end=[]
color=[]
for ind,cols in labels_gene.iterrows():
    hs.append(cols[0])
    st.append(int((cols[1]+cols[2])/2))
    end.append(int((cols[1]+cols[2])/2))
    if cols[0]=="hs1":
        color.append("color=233,105,101")
    elif cols[0]=="hs2":
        color.append("color=112,161,40")
    elif cols[0]=="hs3":
        color.append("color=0,171,183")
    elif cols[0]=="hs4":
        color.append("color=159,122,180")
connector3=pd.DataFrame()

connector3["hs"]=hs
connector3["st"]=st
connector3["end"]=end
connector3["color"]=color

In [ ]:
connector3.to_csv(dir_circos+"connector_4_cancer1.txt",header=False,index=False,sep="\t")

In [ ]:
hs=[]
st=[]
end=[]
color=[]
for ind,cols in labels_gene1.iterrows():
    hs.append(cols[0])
    st.append(int((cols[1]+cols[2])/2))
    end.append(int((cols[1]+cols[2])/2))
    if cols[0]=="hs1":
        color.append("color=233,105,101")
    elif cols[0]=="hs2":
        color.append("color=112,161,40")
    elif cols[0]=="hs3":
        color.append("color=0,171,183")
    elif cols[0]=="hs4":
        color.append("color=159,122,180")
connector4=pd.DataFrame()

connector4["hs"]=hs
connector4["st"]=st
connector4["end"]=end
connector4["color"]=color

In [ ]:
connector4.to_csv(dir_circos+"connector_4_cancer2.txt",header=False,index=False,sep="\t")

In [ ]:
labels_all.to_csv(dir_circos+"labels_all.txt",header=False,index=False,sep="\t")

In [445]:
data_2.head(3)

,cancer,celltype,regulons,cancer_hallmark,genes
BC,BC,Fibroblast,ELAVL1 (3151g),HALLMARK_MITOTIC_SPINDLE,KIF5B
BC,BC,Fibroblast,ELAVL1 (3151g),HALLMARK_MITOTIC_SPINDLE,SMC1A
BC,BC,Fibroblast,ELAVL1 (3151g),HALLMARK_MITOTIC_SPINDLE,RASA2


{'Alveolar',
 'B',
 'Cancer',
 'DC',
 'EC',
 'Epithelial',
 'Fibroblast',
 'Mast',
 'Myeloid',
 'T'}